*Disclaimer: This tutorial is in **FRENCH** only.*

# Tutoriel Jules Vernes


<img src="images/jules_verne_1.png" style='float: left; width: 30%; margin:10%; margin-top:0%; margin-bottom:0%'><img src="images/jules_verne_2.png" style='float: left; width: 30%; margin:10%; margin-top:0%; margin-bottom:0%'>


**Pré-requis :**

- Ce notebook doit avoir été généré avec le template NLP du projet Gabarit.


- Télécharger le fichier `texts.csv` se trouvant ici (https://github.com/OSS-Pole-Emploi/AI_frameworks/tree/main/gabarit/template_nlp/nlp_data) et le placer dans le répertoire `{{package_name}}-data` du package que vous avez généré.


- **Lancer ce notebook avec un kernel utilisant l'environnement virtuel de votre projet**. Pour ce faire, il suffit de faire : `python -m ipykernel install --user --name=your_venv_name` (une fois votre environnement virtuel activé). Evidemment, le projet généré doit être installé sur cet environnement virtuel. Il peut être nécessaire de rafraîchir votre navigateur pour voir le kernel apparaître.

---
---
---

## Table des matières :

<br>  
<br>  


* [1. Objectifs](#objectifs)


* [2. Principes](#principes)


* [3. Entrainement d'un modèle de détection d'auteurs 'out of the box'](#entrainement)


>  * [3.1 Imports et fonctions utilitaires](#imports)

>  * [3.2 Chargement des données](#data)

>  * [3.3 Analyse rapide](#analyse)

>  * [3.4 Découpage du jeu de données en train / test](#decoupage)

>  * [3.5 Preprocessing sur les textes](#preprocessing)

>  * [3.6. Entrainement d'un premier modèle](#model1)

>  * [3.7. Prédictions avec notre premier modèle](#predictions)

>  * [3.8. Création d'un second modèle "maison"](#maison)
>>  * [3.8.1.  Découpage en phrases](#phrases)
>>  * [3.8.2.  Création d'une nouvelle classe de modèle](#new_class)
>>  * [3.8.3.  Packaging de notre nouvelle classe modèle](#packaging)
>>  * [3.8.4.  Réutilisation de notre classe avec les scripts du projet](#scripting)

* [4. It's showtime ](#showtime)


* [5. Conclusion](#conclusion)


---
---
---

## 1. Objectifs <a class="anchor" id="objectifs"></a>




- L'objectif principal de ce tutoriel est de vous introduire au projet Open Source `Gabarit`, développé par l’équipe IA de Pôle Emploi.


- Nous allons traiter un cas d'usage "jouet" : la détection automatique de l'auteur d'un texte du XIXème siècle.


- A la fin de ce tutoriel, nous espérons vous donner tous les outils nécessaires pour démarrer rapidement un nouveau projet d'IA, propre et prêt à être industrialisé.

---
---
---



## 2. Principes <a class="anchor" id="principes"></a>


- Ce projet a été developpé dans une logique d'accélération des développements de nos projets IA, et dans l'objectif de fournir une base de code commune facilitant le passage en industrialisation de nos modèles.


- Pour ce faire, nous proposons différents **templates** de projets IA :  
<br>  
<br>  
    - Template NLP : classification de données textuelles  
<br>  
<br>  
    
    - Template NUM : classification et régression sur des données numériques  
<br>  
<br>  
    
    - Template VISION : classification d'images et détection d'objets  
<br>  
<br>  
    
    
- L'idée n'est pas de créer un outil clé en main 'low-code', mais bien de fournir une base de code que chaque Data Scientist pourra adapter à son projet. Il aura ainsi le contrôle de tout ce qu'il s'y passe.


- Les templates sont très similaires entre eux, même s'ils ont évidemment chacun leur spécificité.  


- Les projets générés sont composés d'une partie 'package' et d'une partie 'scripts' qui permettent notamment de lancer les entrainements.  


- Le projet est construit de manière à ce que tous les modèles soient '**agnostic**'. De cette manière, on appelera un modèle toujours de la même façon (e.g. model.predict(...)) peu importe qu'il s'agisse d'un modèle Sklearn ou TensorFlow.


- Des démonstrateurs Streamlit sont intégrés aux projets pour pouvoir rapidement faire des présentations à votre métier !

---
---
---

## 3. Entrainement d'un modèle de détection d'auteurs 'out of the box' <a class="anchor" id="entrainement"></a>

On va partir de textes de différents auteurs du XIXème siècle et fabriquer un modèle permettant **d'identifier automatiquement son auteur**.

---

### 3.1 Imports et fonctions utilitaires <a class="anchor" id="imports"></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from typing import List
import matplotlib.pyplot as plt
from collections import Counter
from IPython.display import display

from {{package_name}} import utils
from {{package_name}}.preprocessing import preprocess
from {{package_name}}.models_training import utils_models
from {{package_name}}.models_training.model_tfidf_svm import ModelTfidfSvm

---
### 3.2 Chargement des données <a class="anchor" id="data"></a>

In [ ]:
# Certaines fonctions contenues dans utils permettent d'obtenir les chemins vers les répertoires clés du projet
# Ici, on récupère le chemin vers le dossier {{package_name}}-data
data_path = utils.get_data_path()
# On charge les textes pour regarder leur contenu
df = pd.read_csv(os.path.join(data_path, 'texts.csv'), sep=';')

---
### 3.3 Analyse rapide <a class="anchor" id="analyse"></a>

In [ ]:
# Affichage du nombre de lignes / colonnes
n_rows = df.shape[0]
n_columns = df.shape[1]
print(f"Nombre de lignes : {n_rows}")
print(f"Nombre de colonnes : {n_columns}")

In [ ]:
# Affichage de 10 lignes aléatoires
df.sample(10).head(10)

In [ ]:
# Valeurs manquantes
for col in df.columns:
    nb_missing = df[col].isna().sum()
    print(f"Valeurs manquantes pour la colonne \033[1m{col}\033[0m : {nb_missing} -> {round(nb_missing / n_rows * 100, 2)} %")

In [ ]:
# Analyse de la cible
ax = sns.countplot(x=df['author'])
plt.xticks(rotation=45)
plt.show(block=False)

---

### 3.4 Découpage du jeu de données en train / test <a class="anchor" id="decoupage"></a>

Nous allons tout d'abord **découper notre dataset en deux parties**, un ensemble d'entrainement et un ensemble de test afin de pouvoir entraîner et tester notre modèle (nous ne fabriquons pas d'ensemble de validation pour simplifier la présentation). 

Pour cela, nous allons utiliser les scripts contenus dans le package. Les scripts sont des aides pour effectuer les opérations courantes de Data Science. Rien ne vous empêche de créer vos propres scripts !  

<br>  
<br>  

Pour couper en deux le dataset, procédez de la manière suivante :


- Lancez un terminal et naviguez dans le répertoire de votre projet  


- Activez votre environnement virtuel  

    (e.g. sur unix : `source venv_{{package_name}}/bin/activate`)


- Naviguez dans le répertoire `{{package_name}}-scripts/utils`

    ```bash
    cd {{package_name}}-scripts/utils
    ```


- Appelez le script de découpe du dataset pour réaliser une séparation "stratified" : 

    ```bash
    python 0_split_train_valid_test.py -f texts.csv --perc_train 0.6 --perc_valid 0.0 --perc_test 0.4
    ```
    
    Note : les scripts vont automatiquement chercher à charger les fichiers de données, modèles, etc. directement depuis les répertoires du projet prévus à cet effet (e.g. `{{package_name}}-data`, `{{package_name}}-models`, ...)

<br>  
<br>  

Vous pouvez alors voir que deux fichiers, `texts_train.csv` et `texts_test.csv` ont été créés dans `{{package_name}}-data`. 

<br>  
<br>  

On peut regarder à quoi ressemble ce nouveau jeu de données :

In [ ]:
# Comme nous allons le voir, certains jeux de données peuvent être accompagnés d'une première ligne de metadata.
# On conseille donc d'utiliser la fonction utils.read_csv qui va simplement vérifier la présence de cette ligne
df_train, metadata_train = utils.read_csv(os.path.join(data_path, 'texts_train.csv'))
df_test, metadata_test = utils.read_csv(os.path.join(data_path, 'texts_test.csv'))

print('---------------------')
print('------- TRAIN -------')
print('---------------------')
print(f"Métadonnées train : {metadata_train}")
print('Echantillon :')
display(df_train.sample(5).head(5))
print('---------------------')
print('------- TEST -------')
print('---------------------')
print(f"Métadonnées test : {metadata_test}")
print('Echantillon :')
display(df_test.sample(5).head(5))

Si par hasard, vous avez fait une fausse manipulation et que vous devez regénérer les fichiers de train / test, vous pouvez tout simplement ajouter l'option `--overwrite` pour écraser les fichiers erronés.

---

### 3.5 Preprocessing sur les textes <a class="anchor" id="preprocessing"></a>

Nous allons maintenant faire une étape de préprocessing sur les textes afin de les normaliser et de simplifier l'entrainement de nos modèles. Pour cela, nous allons utiliser le script `1_preprocess_data.py` situé dans `{{package_name}}-scripts`.

<br>  
<br>  


Pour préprocesser nos données textuelles :


- Lancez un terminal et naviguez dans le répertoire de votre projet  


- Activez votre environnement virtuel  

    (e.g. sur unix : `source venv_{{package_name}}/bin/activate`)


- Naviguez dans le répertoire `{{package_name}}-scripts`

    ```bash
    cd {{package_name}}-scripts
    ```


- Appelez le script de préprocessing : 

    ```bash
    python 1_preprocess_data.py -f texts_train.csv --input_col text
    ```
    Notes :
    - `--input_col` permet de préciser sur quelle colonne appliquer le preprocessing.
    - On applique le preprocessing uniquement sur le jeu d'entrainement (et le jeu de validation si on en avait un)

<br>  
<br>  

Un nouveau fichier est donc créé dans le répertoire `{{package_name}}-data` : `texts_train_preprocess_P1.csv`.  
Ce fichier est identique au fichier de base, sauf qu'il possède une colonne supplémentaire ('preprocessed_text') qui contient le texte modifié, et une ligne de metadata qui précise quel preprocessing a été appliqué (e.g. `#preprocess_P1`).

Cette métadonnée sera réutilisée par nos modèles pour savoir quel preprocessing doit être appliquée sur une nouvelle entrée en amont de la prédiction.

<br>  
<br>  

On peut regarder à quoi ressemble ce nouveau jeu de données :

In [ ]:
df_train_preprocessed, metadata_train_preprocessed = utils.read_csv(os.path.join(data_path, 'texts_train_preprocess_P1.csv'))
print('------------------------------------')
print('------- TRAIN - PREPROCESSED -------')
print('------------------------------------')
print(f"Métadonnées train - preprocessed : {metadata_train_preprocessed}")
print('Echantillon :')
display(df_train_preprocessed.head(5))

Le préprocessing appliqué est celui fourni de base par le package mais il est très facile d'ajouter ses propres pipelines de preprocessing.

Pour cela :

- Ouvrez le fichier : `{{package_name}}/preprocessing/preprocess.py`.  


- Dans ce module, vous voyez que l'ensemble des pipelines de preprocessing sont renseignées dans la fonction `get_preprocessors_dict`. C'est ici que vous pouvez rajouter vos pipelines personnalisées.  


- On peut également jeter un coup d'oeil à la fonction `preprocess_sentence_P1` qui indique les différentes étapes de préprocessing (pour plus d'informations, il suffit de lire la documentation du package words_n_fun, une librairie facilitant tout le travail de manipulation de texte).

- Si vous avez l'oeil, vous pouvez remarquer que certains accents n'ont pas été traités correctement par la pipeline `preprocess_sentence_P1`. On va donc corriger cela. Pour ce faire, il suffit de rajouter `'remove_accents'` en première étape dans la pipeline de la fonction `preprocess_sentence_P1`. Puis relancez :

    ```bash
    python 1_preprocess_data.py -f texts_train.csv --input_col text
    ```
    Note : Par défaut, ce script écrase les anciennes versions (comportement susceptible de changer dans le futur).

<br>  
<br>  


Les accents sont maintenant correctement traités comme vous pouvez le vérifiez en utilisant la cellule ci-dessous.  


In [ ]:
df_train_preprocessed, metadata_train_preprocessed = utils.read_csv(os.path.join(data_path, 'texts_train_preprocess_P1.csv'))
print('------------------------------------')
print('------- TRAIN - PREPROCESSED -------')
print('------------------------------------')
print(f"Métadonnées train - preprocessed : {metadata_train_preprocessed}")
print('Echantillon :')
display(df_train_preprocessed.head(5))

Notez que, de manière générale, il est **fortement conseillé de créer une nouvelle pipeline de préprocessing plutôt que d'en modifier une existante**.  
En effet, si un de vos anciens modèles utilisait cette pipeline, ses performances vont changer car il utilisera maintenant la nouvelle (qui a le même nom!).  
Comme nous sommes ici au début d'un projet et que nous n'avons pas encore entraîné de modèles, nous pouvons modifier la pipeline de préprocessing.

N.B. : Ce comportement sera certainement amélioré dans les prochaines mises à jour

---

### 3.6. Entrainement d'un premier modèle <a class="anchor" id="model1"></a>

Nous allons maintenant entraîner un premier modèle en utilisant le script `2_training.py`

Pour cela :


- Lancez un terminal et naviguez dans le répertoire de votre projet  


- Activez votre environnement virtuel  

    (e.g. sur unix : `source venv_{{package_name}}/bin/activate`)


- Naviguez dans le répertoire `{{package_name}}-scripts`

    ```bash
    cd {{package_name}}-scripts
    ```


- Appelez le script d'entrainement : 

    ```bash
    python 2_training.py -f texts_train_preprocess_P1.csv --x_col preprocessed_text --y_col author
    ```
    Note :
    - Par défaut, le modèle entrainé sera un simple TF-IDF + SVM sur les données (attention, ça overfit !)
    - Plusieurs autres modèles sont proposés, il suffit d'aller commenter / décommenter les lignes correspondantes dans le fichier d'entrainement `2_training.py`.

<br>  
<br>  

Si vous allez dans `{{package_name}}-models/model_tfidf_svm` vous allez voir un dossier du type `model_tfidf_svm_{YYYY_MM_DD-hh_mm_ss}`. Il s'agit du dossier de sauvegarde du modèle que nous venons d'entrainer, créé automatiquement par le script d'entraînement.  

Vous pouvez par exemple voir le f1_score sur l'ensemble d'entrainement, la matrice de confusion sur l'ensemble d'entraînement etc. Notez que nous aurions pu spécifier un dataset de validation, on aurait ainsi eu accès à des métriques sur ce dataset. 

Ce dossier contient notamment des fichiers .pkl qui vont nous permettre de recharger notre modèle, soit directement par notre package (en mode 'model agnostic'), soit par la librairie du modèle (en mode 'standalone', e.g. Sklearn, TensorFlow, etc.).

---

### 3.7. Prédictions avec notre premier modèle <a class="anchor" id="predictions"></a>

Maintenant que nous avons entrainé un premier modèle, nous voulons le réutiliser pour faire des prédictions sur notre jeu de test !
Nous pourrions utiliser le script `3_predict.py` qui permet de faire des prédictions sur un jeu de test, mais nous allons le faire directement dans ce notebook dans le cadre de la formation.

In [ ]:
# Chargement du modèle entrainé
# TODO : changer {YYYY_MM_DD-hh_mm_ss} pour correspondre au nom de votre modèle ! (cf {{package_name}}-models/model_tfidf_svm)
model, model_conf = utils_models.load_model('model_tfidf_svm_{YYYY_MM_DD-hh_mm_ss}')

Voilà, le modèle est chargé, prêt à être utilisé. Ce que nous allons directement faire sur l'ensemble de test :

In [ ]:
# On commence par chargé le jeu de donnée de test
df_test, _ = utils.read_csv(os.path.join(data_path, 'texts_test.csv'))

# On récupère le preprocessing à partir des configurations du modèle ...
preprocess_str = model_conf['preprocess_str']
preprocessor = preprocess.get_preprocessor(preprocess_str)
# ... et on l'applique à notre jeu de test
df_test['preprocessed_text'] = preprocessor(df['text'])

# On calcul nos prédictions
y_pred = model.predict(df_test['preprocessed_text'], return_proba=False)

# On utilise la fonction inverse_transform pour récupérer un format 'lisible' des prédictions
# Note: ici, dans un cas mono-label, la fonction ne fait rien de particulier
df_test['predictions_1'] = list(model.inverse_transform(np.array(y_pred)))

# Affichage
display(df_test.head(5))

Maintenant qu'on a nos prédictions (qui n'ont pas l'air super ...), on peut évaluer les performances sur notre jeu de test !

In [ ]:
print(f"Accuracy sur le jeu de test : {round((df_test['author'] == df_test['predictions_1']).sum() / df.shape[0] * 100, 2)} %")

Mais, il est tout nul ce modèle !!!

---

### 3.8. Création d'un second modèle "maison" <a class="anchor" id="maison"></a>

Pour remédier à nos mauvais résultats, on pourrait essayer d'autres modèles proposés par notre package dans le script d'entrainement, mais à la place nous allons utiliser une autre manière de faire afin d'illustrer la puissance des frameworks.  
Nous allons **créer notre propre classe de modèle**, qui incorporera tous les fonctionnalités du package que nous avons déjà vu (sauvegarde automatique du modèle, chargement du modèle, calcul des métriques, etc.). 

<br>  
<br>  

La logique que nous allons appliquer est simple. Au lieu d'appliquer un TF-IDF + SVM à des livres entiers, nous allons découper les livres en 'phrases' et faire apprendre le modèle sur ces 'phrases'.  
Pour prédire l'auteur d'un livre, nous allons donc découper en 'phrases', prédire pour chaque 'phrase' un auteur et prendre pour prédiction finale l'auteur qui a le plus de 'phrases' lui correspondant.  

<br>  
<br>  




#### 3.8.1.  Découpage en phrases <a class="anchor" id="phrases"></a>

On doit donc d'abord créer une fonction qui prend un texte et qui le découpe en phrase.  
Comme V0, on va tout simplement enlever quelques ponctuations et découper le texte tous les `nb_word_sentence` mots.  
Un exemple est donné dans le fichier `utils_tutorial_fr.py` (situé au même endroit que ce notebook), mais vous pouvez définir votre propre fonction si vous le souhaitez :

In [ ]:
import utils_tutorial_fr

In [ ]:
def text_to_sentences(text: str, nb_word_sentence: int = 10) -> List[str]:
    '''Transforms a text in sentences.

    Args:
        text (str) : The text to cut in sentences
    Kwargs:
        nb_word_sentence (int) : The number of words in a sentence
    Returns:
        list : A list of sentences
    '''
    return utils_tutorial_fr.text_to_sentences(text, nb_word_sentence)  # A MODIFIER SI VOUS SOUHAITEZ

Testons son fonctionnement :

In [ ]:
text = '''Quelques musiciens accordaient leurs hautbois et leurs déchirants violons, des groupes se formaient
 autour de la tente, et des yeux de paysans se fixaient avec étonnement et volupté sur la grande enseigne où étaient
 écrits en lettres rouges et noires ces mots gigantesques : troupe acrobatique du sieur Pedrillo.
 Plus loin sur un carré de toile peinte l’on distinguait facilement un homme aux formes athlétiques nu comme un sauvage
 et levant sur son dos une quantité énorme de poids. Une banderole tricolore lui sortait de la bouche sur laquelle
 était écrit : Je suis l’Hercule du Nord. Vous dire ce que le pierrot hurla sur son estrade, vous le savez aussi
 bien que moi, certes dans votre enfance vous vous êtes plus d’une fois arrêté devant cette scène grotesque
 et vous avez ri comme les autres des coups de poing et des coups de pied qui viennent à chaque instant
 interrompre l’Orateur au milieu de son discours ou de sa narration. Dans la tente c’était un spectacle
 différent : trois enfants dont le plus jeune avait à peine sept ans, sautaient sur la balustrade intérieure
 de l’escalier, ou bien s’exerçaient sur la corde à la Représentation.'''
text_to_sentences(text, nb_word_sentence=8)



<br>  
<br>  



#### 3.8.2.  Création d'une nouvelle classe de modèle <a class="anchor" id="new_class"></a>

Nous allons créer la classe `ModelAuthor` qui va implémenter notre idée de modèle.  
Pour rappel, on veut toujours faire un modèle TF-IDF + SVM, mais on veut l'appliquer à des phrases et non pas à des textes entiers.
Notre classe va donc descendre de la classe `ModelTfidfSvm`, et on va surcharger certaines fonctions :
- `fit` : avant d'entrainer notre modèle, on veut split le texte en phrases
- `predict` : avant de faire une prédiction, on veut aussi split notre texte en phrases
- `predict_proba` : pareil que predict (à noter qu'ici on a un cas particulier car le SVM ne retourne pas de prédiction)
- `save` : on va rajouter un paramètre à notre modèle, il faut penser le sauvegarder
- `reload_from_standalone` : pareil, quand on veut recharger un modèle 'standalone', il faut rajouter ce nouveau paramètre

<br>  
<br>  

A vous de compléter la classe suivante :

In [ ]:
# INFO : La fonction utils_tutorial_fr.df_texts_to_df_sentences permet de split les textes en phrases
#        et de retourner une dataframe avec toutes les phrases et les auteurs associés.


class ModelAuthor(?????):

    _default_name = 'model_author'  # Utilisé pour le dossier de sauvegarde

    
    def __init__(self, ?????, **kwargs) -> None:
        super().__init__(**kwargs)
        self.????? = ?????  # Paramètre à rajouter : nombre de mots par phrase
        # Dans ce tutoriel, on ne gère pas les cas multilabel
        if self.multi_label:
            raise ValueError("On ne gère pas les cas multilabel")


    def fit(self, x_train, y_train, **kwargs) -> None:
        '''Entrainement du modèle'''
        df_train_sentences = ?????  # Split des textes en phrases avec couples phrases / auteurs
        new_x_train = df_train_sentences['sentence']
        new_y_train = df_train_sentences['author']
        super().fit(new_x_train, new_y_train)

        
    def predict(self, x_test, return_proba: bool = False, **kwargs) -> np.ndarray:
        '''Prediction du modèle'''
        # On process texte par texte
        list_predictions = []
        for text in x_test:
            # On récupère la prédiction sur chaque phrase du texte
            sentences = text_to_sentences(text, nb_word_sentence=?????)
            predictions = super().predict(sentences)
            # On compte le nombre d'occurrences
            counter_author_predictions = dict(Counter(list(predictions)))

            # Si on ne retourne pas de probabilités, on incrémente juste la liste de prédictions
            if not return_proba:
                predicted_author = max(counter_author_predictions, key=counter_author_predictions.get)
                ??????  # On ajoute l'auteur predit à la liste
            else:
                # On va considérer les probabilités comme le pourcentage de phrases attribuées à chaque auteur
                nb_sentences = len(sentences)
                list_probas = [counter_author_predictions.get(author, 0) / nb_sentences for author in self.list_classes]
                list_predictions.append(list_probas)
        
        # Return
        return np.array(list_predictions)

    
    def predict_proba(self, x_test, **kwargs) -> np.ndarray:
        return self.predict(x_test, return_proba=?????, **kwargs)
    
    
    def save(self, json_data=None) -> None:
        '''Sauvegarde du modèle'''
        if json_data is None:
            json_data = {}
        json_data['?????'] = self.?????
        # Save
        super().save(json_data=json_data)
        
    
    def reload_from_standalone(self, **kwargs) -> None:
        '''Recharge un modèle à partir de sa sauvegarde standalone et de son fichier de conf'''
        super().reload_from_standalone(**kwargs)
        configuration_path = kwargs.get('configuration_path', None)
        with open(configuration_path, 'r', encoding='{{default_encoding}}') as f:
            configs = json.load(f)
        self.????? = configs.get('trained', self.?????)


Testons cette nouvelle classe !

In [ ]:
# Entrainement
df_train_preprocessed, metadata_train_preprocessed = utils.read_csv(os.path.join(data_path, 'texts_train_preprocess_P1.csv'))
model = ModelAuthor(nb_word_sentence=100)
x_train, y_train = df_train_preprocessed['preprocessed_text'], df_train_preprocessed['author']
model.fit(x_train, y_train)

# Sauvegarde
preprocess_str = metadata_train_preprocessed.replace('#', '')
model.save({'preprocess_str': preprocess_str})

# Prédictions sur tests
df_test, _ = utils.read_csv(os.path.join(data_path, 'texts_test.csv'))
preprocessor = preprocess.get_preprocessor(preprocess_str)
x_test = preprocessor(df_test['text'])
predictions = model.predict(x_test)
probas = model.predict_proba(x_test)
df_results = pd.DataFrame({
    'text': df_test['text'],
    'true_author': df_test['author'],
    'predicted_author': predictions,
    'proba': probas.max(axis=1)
})
display(df_results)

# Evaluations
y_pred_train = model.predict(x_train, return_proba=False)
model.get_and_save_metrics(y_train, y_pred_train, type_data='train')
model.get_and_save_metrics(df_test['author'], predictions, type_data='test')

On voit qu'on est bien meilleur qu'avant sur notre jeu de test ! Super !

Si vous voulez, vous pouvez vérifier dans votre répertoire de modèles que le nouveau modèle apparait bien (dans un sous dossier `model_author`).


<br>  
<br>  

Ainsi en moins d'une centaine de ligne de code, nous avons **développé une nouvelle classe de modèle** qui va pouvoir utiliser tous les outils du projet.

<br>  
<br>  



#### 3.8.3.  Packaging de notre nouvelle classe modèle <a class="anchor" id="packaging"></a>

Maintenant qu'on a notre nouvelle classe de modèle, on a envie de **l'intégrer à notre package**.  
Cela va nous permettre d'utiliser tous les scripts mis à disposition, d'ajouter des tests unitaires pour vérifier le bon fonctionnement de notre classe, et de simplifier l'industrialisation de nos futurs modèles.


<br>  
<br>  

Pour ce faire, il va falloir encapsuler notre classe dans un fichier python.  
Pour accélérer la présentation, nous l'avons déjà effectué pour vous dans le fichier `model_author.py` situé dans le même dossier que ce notebook. Libre à vous de l'éditer si vous voulez ajouter vos spécificités (notamment sur le split en phrases).


<br>  
<br>  


Pour incorporer notre nouvelle classe, il faut :

- Copier le fichier `model_author.py` à l'endroit où se trouve tous les autres classes de modèles : `{{package_name}}/models_training/`


- Modifier les imports du script `2_training.py` pour rajouter notre classe de modèle :

    ```python
    from {{package_name}}.models_training.model_author import ModelAuthor
    ```

- Ajouter un exemple d'instanciation dans le script `2_training.py` (cf. tous les exemples commentés au milieu du script) :

    ```python
    model = ModelAuthor(x_col=x_col, y_col=y_col, 
                        level_save=level_save,
                        multi_label=multi_label,
                        nb_word_sentence=100)
    ```

- (NE PAS FAIRE POUR LE TUTO) : Ajouter l'import dans 0_reload_model.py et mettre à jour le dictionnaire `model_type_dicts`


- (NE PAS FAIRE POUR LE TUTO) : Développer les TUs associés à notre classe (à grand coups de copier-collers !)



<br>  
<br>  



#### 3.8.4.  Réutilisation de notre classe avec les scripts du projet <a class="anchor" id="scripting"></a>

On va réentrainer notre nouveau modèle directement avec les scripts du projet.  


Pour ce faire, il vous suffit de commenter le modèle TF-IDF + SVM et de décommenter le modèle 'Author' dans `2_training.py` (le bout de code que vous venez de rajouter), et de relancer les mêmes commandes que précedemment :

- Entrainement : `python 2_training.py -f texts_train_preprocess_P1.csv --x_col preprocessed_text --y_col author`  


- Prédictions : `python 3_predict.py -f texts_test.csv --x_col text --y_col author -m model_author_{YYYY_MM_DD-hh_mm_ss}`  

    Note : il faut changer le nom du modèle par le vôtre.


<br>  
<br>  


Maintenant vous pouvez vous amusez à changer les paramètres de notre nouveau modèle (nombre de mots par phrases, paramètres tfidf (cf. classe `ModelTfidfSvm`), etc.) pour essayer d'obtenir le meilleur score possible sur le jeu de test !

Attention de ne pas overfitter dessus ! 😅

---
---
---

## 4. It's showtime ! <a class="anchor" id="showtime"></a>

Nous allons finalement utiliser un autre outil incorporé dans les frameworks : le démonstrateur.  

Il est souvent nécessaire de montrer le fonctionnement d'un modèle IA à votre métier, mais aussi à vos pairs.  
Nous avons donc inclus un démonstrateur Streamlit à nos projets pour vous permettre de démontrer très rapidement les performances de vos meilleurs modèles !

Pour le lancer, il vous suffit de :


- Lancez un terminal et naviguez dans le répertoire de votre projet  


- Activez votre environnement virtuel  

    (e.g. sur unix : `source venv_{{package_name}}/bin/activate`)


- Naviguez dans le répertoire `{{package_name}}-scripts`

    ```bash
    cd {{package_name}}-scripts
    ```


- Appelez le démonstrateur streamlit : 

    ```bash
    streamlit run 4_demonstrator.py
    ```
    Note : par défaut streamlit se lance sur le port 8501 de votre localhost : http://localhost:8501

<br>  
<br>  


Dans le menu de gauche, vous pouvez sélectionner le modèle que vous voulez utiliser, soit le premier modèle TF-IDF + SVM que nous avons entrainé, soit un de nos nouveaux modèles basé sur le découpage en phrases.  
Il suffit ensuite d'entrer le texte que vous voulez tester et appuyer sur predict pour accéder à la prédiction du modèle (bon idéalement il faudrait copier un livre entier 🙄 vous pouvez cocher 'Multiple lines').


Note : vous pouvez évidemment adapter ce démonstrateur à vos besoin en éditant le fichier `4_demonstrator.py`.



---
---
---

## 5. Conclusion <a class="anchor" id="conclusion"></a>


Ce petit exemple illustre la manière d'utiliser les frameworks :

- On part d'un projet contenant tous les outils pour développer rapidement un modèle.  


- On teste quelques modèles inclus pour voir si un correspond à notre cas d'usage.  


- Si besoin, on développe son propre modèle en héritant des classes définies dans le projet pour profiter de toutes les fonctions incluses dedans.


<br>  
<br>  



Pour la partie industrialisation, nous vous invitons à consulter notre Github.  

L'idée globale est : 
- Uploader les modèles sur un système de stockages d'artefacts  


- Build votre package et l'uploader (e.g. sur PyPI)  


- Créer un autre projet API qui import votre package  


- Télécharger votre modèle à l'initialisation du service API et exposer une fonction `predict`